In [70]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import scipy
import plotly.express as px

import re

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import re

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [71]:
# sentiment = pd.read_csv('/home/jupyter/mnt/s3/hack-data/hse/sentiment.csv')

In [72]:
# banki = pd.read_csv('/home/jupyter/mnt/s3/hack-data/hse/banki.csv')

In [73]:
# bank_sentiment = pd.read_csv('/home/jupyter/mnt/s3/hack-data/hse/bank-sentiment.csv')

In [74]:
train = pd.read_csv('/home/jupyter/mnt/s3/hack-data/hse/train.csv', index_col=0)

In [75]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove non-alphabetic characters
    text = re.sub(r'[^а-яА-Я\s]', '', text)
    
    # Tokenize text
    tokens = word_tokenize(text, language='russian')
    
    # Remove stop words
    stop_words = set(stopwords.words('russian'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stem tokens
    stemmer = SnowballStemmer('russian')
    tokens = [stemmer.stem(token) for token in tokens]
    
    # Join tokens back into a string
    text = ' '.join(tokens)
    
    return text

In [76]:
train['clean_text'] = train['sentence'].apply(preprocess_text)

In [77]:
train['1category'].unique()

array(['Communication', '?', 'Quality', 'Price', 'Safety'], dtype=object)

In [78]:
train['2category'].unique()

array([nan, 'Quality', 'Communication', 'Price', 'Safety'], dtype=object)

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

In [80]:
# Fill nan values
train.isnull().sum()

sentence          0
1category         0
2category     18362
sentiment         0
clean_text        0
dtype: int64

In [81]:
train = train.fillna('?')

In [82]:
train.isnull().sum()

sentence      0
1category     0
2category     0
sentiment     0
clean_text    0
dtype: int64

In [83]:
train

,sentence,1category,2category,sentiment,clean_text
4754,При этом всегда получал качественные услуги.,Communication,?,+,получа качествен услуг
4417,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",?,?,−,виж хот постав сервис
3629,"Вот так ""Мой любимый"" банк МКБ меня обманул.",?,?,−,любим банк мкб обманул
11640,Отвратительное отношение к клиентам.,Communication,?,−,отвратительн отношен клиент
5571,"Всегда в любое время дня и ночи помогут, ответ...",Communication,?,+,люб врем дня ноч помогут ответ решат
...,...,...,...,...,...
8004,Никогда и ни в коем случае не открывайте счет ...,Communication,?,−,ко случа открыва счет недостойн довер банк кот...
18182,ТИ откровенно забили на качество и развивают с...,Quality,?,−,ти откровен заб качеств развива сво мег классн...
744,"Я считаю, это прорыв и лидерство финансовых ус...",?,?,+,счита эт прор лидерств финансов услуг централь...
6220,"Писал мужчина очень доходчиво, не финансовым я...",Communication,?,+,писа мужчин очен доходчив финансов язык работа...


In [84]:
columns_onehot = ['1category', '2category']

In [85]:
ctransformer = ColumnTransformer([("encoded_data", OneHotEncoder(handle_unknown='ignore', sparse=False), columns_onehot),])
ct_encoded_results = ctransformer.fit_transform(train)
# Get Feature Names of Encoded columns
ctransformer.get_feature_names()

['encoded_data__x0_?',
 'encoded_data__x0_Communication',
 'encoded_data__x0_Price',
 'encoded_data__x0_Quality',
 'encoded_data__x0_Safety',
 'encoded_data__x1_?',
 'encoded_data__x1_Communication',
 'encoded_data__x1_Price',
 'encoded_data__x1_Quality',
 'encoded_data__x1_Safety']

In [86]:
df_ct_encoded_data = pd.DataFrame(ct_encoded_results, columns=ctransformer.get_feature_names())
# Viewing first few rows of data

df = pd.concat([train.reset_index(drop=True), df_ct_encoded_data.reset_index(drop=True)], axis=1)
# Dropping drive-wheels, make and engine-location columns as they are encoded
df.drop(columns_onehot, inplace=True, axis=1)
# Viewing few rows of data

In [87]:
train

,sentence,1category,2category,sentiment,clean_text
4754,При этом всегда получал качественные услуги.,Communication,?,+,получа качествен услуг
4417,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",?,?,−,виж хот постав сервис
3629,"Вот так ""Мой любимый"" банк МКБ меня обманул.",?,?,−,любим банк мкб обманул
11640,Отвратительное отношение к клиентам.,Communication,?,−,отвратительн отношен клиент
5571,"Всегда в любое время дня и ночи помогут, ответ...",Communication,?,+,люб врем дня ноч помогут ответ решат
...,...,...,...,...,...
8004,Никогда и ни в коем случае не открывайте счет ...,Communication,?,−,ко случа открыва счет недостойн довер банк кот...
18182,ТИ откровенно забили на качество и развивают с...,Quality,?,−,ти откровен заб качеств развива сво мег классн...
744,"Я считаю, это прорыв и лидерство финансовых ус...",?,?,+,счита эт прор лидерств финансов услуг централь...
6220,"Писал мужчина очень доходчиво, не финансовым я...",Communication,?,+,писа мужчин очен доходчив финансов язык работа...


In [89]:
df.head()

,sentence,sentiment,clean_text,encoded_data__x0_?,encoded_data__x0_Communication,encoded_data__x0_Price,encoded_data__x0_Quality,encoded_data__x0_Safety,encoded_data__x1_?,encoded_data__x1_Communication,encoded_data__x1_Price,encoded_data__x1_Quality,encoded_data__x1_Safety
0,При этом всегда получал качественные услуги.,+,получа качествен услуг,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",−,виж хот постав сервис,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,"Вот так ""Мой любимый"" банк МКБ меня обманул.",−,любим банк мкб обманул,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Отвратительное отношение к клиентам.,−,отвратительн отношен клиент,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,"Всегда в любое время дня и ночи помогут, ответ...",+,люб врем дня ноч помогут ответ решат,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
